In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set()

from sklearn.preprocessing import MultiLabelBinarizer

In [2]:
X = pd.read_csv('src/deepwalk_emb/karate_32.csv', delim_whitespace=True, header=None, skiprows=1, index_col=0).sort_index()
X.index.name = None
y = pd.read_csv('data/graphs/karate.mylabels', header=None)[0]

In [3]:
X = pd.read_csv(
    'src/deepwalk_emb/blog_dim128.csv',
    delim_whitespace=True, header=None,
    skiprows=1,
    index_col=0
).sort_index()
X.index.name=None

In [4]:
y = pd.read_csv(
    'data/graphs/BlogCatalog/data/group-edges.csv', 
    header=None, 
    delim_whitespace=True, 
    index_col=0
).sort_index()
y.index.name = None

yy = []
for i in range(1, 10313):
    values = y.loc[i].values
    if len(values) != 1:
        values = [item[0] for item in values]
    else:
        values = list(values)
    yy.append(values)
    
y_multilabel = MultiLabelBinarizer().fit_transform(yy)

In [5]:
X.loc[6858:6859]

,1,2,3,4,5,6,7,8,9,10,...,119,120,121,122,123,124,125,126,127,128
6858,-0.295763,0.798570,0.326575,-0.148498,-0.364958,-0.20715,0.210560,0.501065,1.006918,-0.290854,...,-0.237573,-0.190263,0.226695,-0.175909,-0.332940,-0.509273,0.450450,-0.232533,0.001708,0.411823
6859,-0.022822,-0.084983,0.022788,0.085154,-0.012066,-0.10226,-0.140816,0.004663,-0.286486,-0.076519,...,-0.150373,0.019662,0.046096,-0.320828,0.176368,-0.003524,-0.286341,0.044632,-0.288565,0.082986


In [6]:
y_multilabel[6857]

array([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0])

In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from src.classifiers import MultilabelOVRClassifier

from sklearn.model_selection import cross_val_score, ShuffleSplit

In [8]:
clf1 = LogisticRegression(C=100)

In [9]:
clf2 = OneVsRestClassifier(LogisticRegression(C=1, verbose=0), n_jobs=-1)

In [10]:
MultilabelOVRClassifier.set_labels(X.index.values, y_multilabel)
clf3 = MultilabelOVRClassifier(LogisticRegression(C=1, verbose=0), n_jobs=-1)

In [11]:
clf = clf3

In [12]:
clf.fit(X, y_multilabel)

MultilabelOVRClassifier(estimator=LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
            n_jobs=-1)

In [13]:
pred = clf.predict(X)

In [17]:
pred[0]

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])

In [ ]:
len(X)

In [18]:
len([_ for arr in clf.predict(X) if sum(arr)>1])

10312

In [ ]:
[sorted(item) for item in clf.predict_proba(X.loc[1:7])]

In [19]:
ss = ShuffleSplit(n_splits=3, random_state=42, train_size=0.6)

In [20]:
scores = cross_val_score(clf, X, y_multilabel, cv=ss.split(X), scoring='f1_micro', verbose=1)
print(scores)
print(scores.mean())

[ 0.33471718  0.32503809  0.33891213]
0.33288913375


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:   34.9s finished


In [29]:
scores = cross_val_score(clf, X, y_multilabel, cv=3, scoring='f1_macro', verbose=1, n_jobs=-1)
print(scores)
print(scores.mean())

/home/stas/anaconda3/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py:540: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/home/stas/anaconda3/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py:540: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/home/stas/anaconda3/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py:540: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)


[ 0.21120028  0.2047038   0.17524493]
0.197049667036


[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:  1.0min finished
